In [1]:
import numpy as np
from glue_jupyter.bqplot.scatter import BqplotScatterView
from glue_jupyter import JupyterApplication
from glue.core import Data

app = JupyterApplication()

# Setup some random data
def add_random_data():
    rand_data = Data(distance=np.random.sample(10) * 2e6,
                     velocity=np.random.sample(10) * 1000,
                     class_id=[np.random.randint(0, 3) for _ in range(10)],
                     student_id=np.arange(10) + 1,
                     galaxy_type=[['spiral', 'elliptical', 'irregular'][np.random.randint(0, 3)] for _ in range(10)],
                     label='random_data' + str(len(app.data_collection)))
    app.data_collection.append(rand_data)
    
add_random_data()

# This would be equivalent to the items in `app._viewer_handlers`
viewer = app.new_data_viewer(
    BqplotScatterView,
    show=True)

viewer.add_data('random_data0')

data = app.data_collection[0]
viewer.state.x_att = data.id['distance']
viewer.state.y_att = data.id['velocity']

# This is equivalent to the an item in `app.viewers`
figure = viewer.figure_widget

/Users/nmearl/.pyenv/versions/3.9.1/envs/cosmicds/lib/python3.9/site-packages/glue/external/echo/__init__.py:3: UserWarning: glue.external.echo is deprecated, import from echo directly instead
  warnings.warn('glue.external.echo is deprecated, import from echo directly instead')


LayoutWidget(controls={'toolbar_selection_tools': BasicJupyterToolbar(tools_data={'bqplot:home': {'tooltip': '…

In [3]:
DataControls()
# app.data_collection.subset_groups[0].subsets[0]

DataControls(cls_items=[0, 1, 2], events=['class_selected', 'galaxy_selected'], gal_items=['spiral', 'elliptic…

In [2]:
from ipyvuetify import VuetifyTemplate
from traitlets import Unicode, List

class DataControls(VuetifyTemplate):
    template = Unicode("""
<template>
  <v-container fluid>
    <v-row>
      <v-col cols="6">
        <v-combobox
          v-model="gal_select"
          :items="gal_items"
          label="Galaxy Type"
          multiple
          outlined
          dense
          @change="galaxy_selected"
        ></v-combobox>
      </v-col>
      <v-col cols="6">
        <v-combobox
          v-model="cls_select"
          :items="cls_items"
          label="Class ID"
          multiple
          outlined
          dense
          @change="class_selected"
        ></v-combobox>
      </v-col>
    </v-row>
  </v-container>
</template>
    """).tag(sync=True)
    
    gal_items = List(['spiral', 'elliptical', 'irregular']).tag(sync=True)
    gal_select = List([]).tag(sync=True)
    
    cls_items = List(set(app.data_collection[0]['class_id'])).tag(sync=True)
    cls_select = List([]).tag(sync=True)
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
        self.gal_state = None
        self.cls_state = None
        
    def update_subset_state(self):
        for ssgrp in app.data_collection.subset_groups:
            app.data_collection.remove_subset_group(ssgrp)
            
        if self.gal_state is None and self.cls_state is None:
            return
        
        if self.gal_state is not None and self.cls_state is None:
            state = self.gal_state
        elif self.gal_state is None and self.cls_state is not None:
            state = self.cls_state
        else:
            state = self.gal_state & self.cls_state
        
        label = 'Selected'
        subset_group = app.data_collection.new_subset_group(label, state)
        subset_group.style.color = '#ff0000'
        subset_group.style.alpha = 1
        
    def vue_galaxy_selected(self, event):
        data = app.data_collection[0]
        if len(event) > 0:
            self.gal_state = np.bitwise_or.reduce([data.id['galaxy_type'] == x for x in event])
        else:
            self.gal_state = None
        self.update_subset_state()
        
    def vue_class_selected(self, event):
        data = app.data_collection[0]
        if len(event) > 0:
            self.cls_state = np.bitwise_or.reduce([data.id['class_id'] == x for x in event])
        else:
            self.cls_state = None
        self.update_subset_state()
    